# Lesson 07 — A2A Client Fundamentals

Build a client that **discovers** and **communicates** with A2A agents.

## What You'll Learn

- Resolve an Agent Card with `A2ACardResolver`
- Send blocking requests with `A2AClient.send_message()`
- Stream responses with `A2AClient.send_message_streaming()`
- Parse Tasks, Messages, and Parts
- Handle errors gracefully

## Prerequisites

- **Lesson 06 server must be running** on `http://localhost:10001`
- Run: `cd ../06-a2a-server/src && python server.py`

> **A2A SDK docs:** [pypi.org/project/a2a-sdk](https://pypi.org/project/a2a-sdk/)


## Step 1 — Install Dependencies


In [4]:
# ── Dependencies ──────────────────────────────────────────────────────────────
# a2a-sdk[http-server] → A2A Python SDK (includes A2AClient, A2ACardResolver)
# httpx                → Async HTTP client used directly in this lesson
# python-dotenv        → Loads environment variables from .env
#
# If you have the venv active (a2a-examples kernel selected) these are already
# installed. Run this cell anyway to ensure the kernel is up to date.
# ─────────────────────────────────────────────────────────────────────────────
%pip install "a2a-sdk[http-server]" httpx python-dotenv --quiet
print("✅ Dependencies ready")

Note: you may need to restart the kernel to use updated packages.
✅ Dependencies ready



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dotenv import find_dotenv, load_dotenv

# ── Load .env ─────────────────────────────────────────────────────────────
# The A2A client (this notebook) does NOT call GitHub Models directly —
# it talks to the A2A server, which holds the GITHUB_TOKEN.
# We still load .env here for consistency and future lessons.
#
# Reminder: The Lesson 06 server must be running on localhost:10001
#   cd _examples/a2a
#   .\scripts\run_server.ps1     # Windows
#   bash scripts/run_server.sh   # Linux / macOS
# ─────────────────────────────────────────────────────────────────────────────
env_path = find_dotenv(raise_error_if_not_found=False)
if env_path:
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path}")
else:
    print("ℹ  No .env found — that is OK for the client notebook")

print()
print("⚠  This notebook requires the Lesson 06 server running on http://localhost:10001")
print("   Start it: .\\scripts\\run_server.ps1  (or bash scripts/run_server.sh)")

✅ Loaded .env from: y:\.sources\localm-tuts\a2a\_examples\.env

⚠  This notebook requires the Lesson 06 server running on http://localhost:10001
   Start it: .\scripts\run_server.ps1  (or bash scripts/run_server.sh)


## Step 2 — Discover the Agent

Every A2A agent serves its Agent Card at `/.well-known/agent.json`.
The `A2ACardResolver` fetches and parses this automatically.

This is the **discovery** step — your client learns the agent's
name, skills, supported modes, and capabilities.


In [6]:
import httpx
from a2a.client import A2ACardResolver

BASE_URL = "http://localhost:10001"

# timeout=60.0 — GitHub Models can take 20-40s; default httpx timeout (5s) is too short
async with httpx.AsyncClient(timeout=60.0) as httpx_client:
    resolver = A2ACardResolver(
        httpx_client=httpx_client,
        base_url=BASE_URL,
    )
    agent_card = await resolver.get_agent_card()

print(f"Agent Name:    {agent_card.name}")
print(f"Description:   {agent_card.description}")
print(f"Version:       {agent_card.version}")
print(f"Streaming:     {agent_card.capabilities.streaming}")
print(f"Skills:        {[s.name for s in agent_card.skills]}")


Agent Name:    QAAgent
Description:   Answers questions about insurance policies using GitHub Phi-4
Version:       1.0.0
Streaming:     True
Skills:        ['Policy Question Answering']


## Step 3 — Create the A2A Client

The `A2AClient` wraps httpx and provides typed methods for
sending messages. It needs the Agent Card (from discovery)
and an httpx client.


In [7]:
import warnings

from a2a.client import A2AClient
warnings.filterwarnings("ignore", category=DeprecationWarning)  # suppress A2AClient deprecation notice

# timeout=60.0 — keep consistent with the discovery client above
httpx_client = httpx.AsyncClient(timeout=60.0)

client = A2AClient(
    httpx_client=httpx_client,
    agent_card=agent_card,
)

print(f"\u2705 A2AClient ready \u2014 targeting {agent_card.url}")
print("\u2139  A2AClient is deprecated in newer SDK versions; ClientFactory is the replacement.")
print("   This lesson uses A2AClient for simplicity \u2014 the concepts are identical.")


✅ A2AClient ready — targeting http://localhost:10001/
ℹ  A2AClient is deprecated in newer SDK versions; ClientFactory is the replacement.
   This lesson uses A2AClient for simplicity — the concepts are identical.


## Step 4 — Helper: Build a Message Request

Every A2A message follows JSON-RPC 2.0 format:

```json
{
  "jsonrpc": "2.0",
  "id": "<unique-id>",
  "method": "message/send",
  "params": {
    "message": {
      "role": "user",
      "parts": [{ "kind": "text", "text": "..." }],
      "messageId": "<unique-id>"
    }
  }
}
```

The SDK provides `SendMessageRequest` and `MessageSendParams` to construct this.


In [8]:
from uuid import uuid4
from a2a.types import MessageSendParams, SendMessageRequest


def build_request(question: str) -> SendMessageRequest:
    """Build a blocking SendMessageRequest for the given question."""
    payload = {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": question}],
            "messageId": uuid4().hex,
        }
    }
    return SendMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(**payload),
    )


print("✅ build_request() helper defined")

✅ build_request() helper defined


## Step 5 — Blocking Call (message/send)

A blocking call waits for the agent to finish processing and
returns the complete response. The response is a `Task` object
with status, messages, and artifacts.


In [9]:
request = build_request("What is the annual deductible?")

response = await client.send_message(request)

# In a2a-sdk v0.3.x, message/send returns a Message directly (not a Task).
# The typed result lives at response.root.result.
# Note: pydantic fields are snake_case (message_id, not messageId)
msg = response.root.result

print(f"Message ID:  {msg.message_id}")
print(f"Role:        {msg.role.value}")
print(f"Kind:        {msg.kind}")
print()

# Each Part is a discriminated union wrapper — access content via part.root
for part in msg.parts:
    if part.root.kind == "text":
        print(f"Agent says: {part.root.text}")


Message ID:  2669ce82-074e-455d-b056-a332a12cd9d0
Role:        agent
Kind:        message

Agent says: The policy document does not specify an "annual deductible" as a separate category. Instead, it lists specific deductibles for different types of services:

- Standard Plan Deductible: $500 per incident
- Emergency Room Deductible: $250 per visit
- Prescription Drug Deductible: $100 per year

These are the deductibles applicable to different services under the policy. If you are referring to a total annual deductible amount, it is not explicitly stated in the document. 

For more detailed information, you might want to contact ACME Insurance directly through their customer service or claims department.


## Step 6 — Parse the Response Helper

Let's create a reusable function to extract text from any A2A response.


In [10]:
def extract_text(response) -> str:
    """Extract text from a Message response (a2a-sdk v0.3.x).

    In v0.3.x, message/send returns a Message at response.root.result.
    Each part is a discriminated union accessed via part.root.
    """
    try:
        msg = response.root.result
    except AttributeError:
        return f"Error: unexpected response shape \u2014 {type(response).__name__}"

    texts = []
    for part in msg.parts:
        if part.root.kind == "text":
            texts.append(part.root.text)

    return "\n".join(texts) if texts else "(no text in response)"


print("\u2705 extract_text() helper defined")


✅ extract_text() helper defined


## Step 7 — Ask Multiple Questions

Test the agent with several questions to verify it's answering
from the knowledge base.


In [11]:
questions = [
    "What is the monthly premium?",
    "Are cosmetic procedures covered?",
    "How do I file a claim?",
    "What is the meaning of life?",   # not in knowledge base
]

for q in questions:
    request = build_request(q)
    response = await client.send_message(request)
    answer = extract_text(response)
    print(f"Q: {q}")
    print(f"A: {answer}")
    print("-" * 60)

Q: What is the monthly premium?
A: The monthly premium for the Standard Plan is $150. This information is found in the "COVERAGE SUMMARY" section of the policy document.
------------------------------------------------------------
Q: Are cosmetic procedures covered?
A: Cosmetic procedures are generally not covered under the ACME Insurance Standard Policy. The policy document explicitly states that cosmetic procedures are excluded unless they are medically necessary. Here is the relevant section:

**EXCLUSIONS**
- Cosmetic procedures (unless medically necessary)

If a cosmetic procedure is deemed medically necessary, it may be covered, but otherwise, it is excluded from coverage.
------------------------------------------------------------
Q: How do I file a claim?
A: To file a claim with ACME Insurance under the Standard Policy, you should follow these steps:

1. **Submit Claims Within 90 Days**: Ensure that you submit your claim within 90 days of the service date. This is crucial to m

## Step 8 — Streaming Call (message/stream)

Streaming returns partial results as Server-Sent Events (SSE).
This is useful for real-time UI updates.

The `SendStreamingMessageRequest` is identical to `SendMessageRequest`
except the method is `message/stream`.


In [12]:
from a2a.types import SendStreamingMessageRequest


def build_streaming_request(question: str) -> SendStreamingMessageRequest:
    """Build a streaming request for the given question."""
    payload = {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": question}],
            "messageId": uuid4().hex,
        }
    }
    return SendStreamingMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(**payload),
    )


streaming_request = build_streaming_request("Explain the claims process step by step.")

print("Streaming response:")
print("=" * 60)

async for event in client.send_message_streaming(streaming_request):
    event_type = type(event).__name__
    print(f"  Event: {event_type}")

    # Navigate the discriminated-union response tree
    root = getattr(event, "root", event)
    result = getattr(root, "result", None)
    if result is None:
        continue

    # v0.3.x: result may be a Message or have a status.message (Task-style)
    if hasattr(result, "parts"):
        # Direct Message
        for part in result.parts:
            pr = getattr(part, "root", part)
            if getattr(pr, "kind", None) == "text":
                preview = pr.text[:120].replace("\n", " ")
                print(f"  Text: {preview}{'...' if len(pr.text) > 120 else ''}")
    elif hasattr(result, "status") and result.status and result.status.message:
        # Task-style (older SDK / future versions)
        for part in result.status.message.parts:
            pr = getattr(part, "root", part)
            if getattr(pr, "kind", None) == "text":
                preview = pr.text[:120].replace("\n", " ")
                print(f"  Text: {preview}{'...' if len(pr.text) > 120 else ''}")

print("=" * 60)
print("\u2705 Streaming complete")


Streaming response:
  Event: SendStreamingMessageResponse
  Text: Certainly! Here is the step-by-step claims process as outlined in the policy document:  1. **Submit Claims Within 90 Day...
✅ Streaming complete


## Step 9 — Error Handling

A2A uses JSON-RPC error codes. Common errors:

| Code   | Meaning             |
| ------ | ------------------- |
| -32700 | Parse error         |
| -32600 | Invalid request     |
| -32601 | Method not found    |
| -32001 | Task not found      |
| -32002 | Task not cancelable |

Let's handle errors properly:


In [13]:
async def safe_query(client: A2AClient, question: str) -> str:
    """Send a question and handle errors gracefully."""
    try:
        request = build_request(question)
        response = await client.send_message(request)

        # Check for JSON-RPC error response (error object at root level)
        root = getattr(response, "root", None)
        if root is not None and hasattr(root, "error") and root.error:
            return f"[Error {getattr(root.error, 'code', '?')}] {getattr(root.error, 'message', root.error)}"

        return extract_text(response)

    except httpx.ConnectError:
        return "[Connection Error] Is the server running on localhost:10001?"
    except Exception as e:
        return f"[Unexpected Error] {type(e).__name__}: {e}"


# Test it
answer = await safe_query(client, "What medications are excluded?")
print(f"Answer: {answer}")


Answer: The policy document does not specifically list excluded medications. However, it does mention that "Experimental treatments not FDA-approved" are excluded. This implies that medications that are experimental and not approved by the FDA would not be covered under this policy. For more specific information about excluded medications, you may need to contact ACME Insurance directly or review the detailed policy documents or exclusions section that might be available upon request. 

If you have any other questions or need further clarification, feel free to ask!


## Step 10 — Experiment!

Try your own questions below. The agent can only answer based
on the ACME Insurance policy document loaded in the server.


In [14]:
# Change this question and re-run the cell
my_question = "What happens if I miss a premium payment?"

answer = await safe_query(client, my_question)
print(f"Q: {my_question}")
print(f"A: {answer}")

Q: What happens if I miss a premium payment?
A: The provided policy document does not contain specific information regarding the consequences of missing a premium payment. It is important to refer to the full policy terms or contact ACME Insurance directly for detailed information on their procedures for missed payments. You can reach them at 1-800-555-ACME or through their online portal at https://portal.acme-insurance.example.com for further assistance.


## Cleanup


In [15]:
await httpx_client.aclose()
print("✅ httpx client closed")

✅ httpx client closed


## Summary

| Concept              | What You Learned                                   |
| -------------------- | -------------------------------------------------- |
| **Discovery**        | `A2ACardResolver` fetches the Agent Card           |
| **Blocking calls**   | `send_message()` returns a complete Task           |
| **Streaming**        | `send_message_streaming()` yields partial results  |
| **Response parsing** | Navigate Task → status → message → parts           |
| **Error handling**   | Check `response.error` + catch connection failures |

## Next Steps

You've completed the full A2A loop: **Agent → Server → Client**.

In Lesson 08, you'll build a more complex multi-step agent using
Google's Agent Development Kit (ADK) — and expose it over A2A.

→ Continue to [Lesson 08 — Health Research Agent (ADK)](../08-health-research-adk/)
